In [1]:
def get_hz_py_dic():
    py_hz_file = "pinyin2hanzi.txt"
    hz_py_dic = dict()
    py_dic = dict()
    hz_list = []
    py_list = []  
    py_hz_dic = dict()
    with open(py_hz_file, 'r', encoding='utf8') as fin:
        i = -1
        j = 0
        for line in fin.readlines():
            py, hz = line.split(' ')
            hzs = hz.strip('\n').split(' ')
            if(j == 0):
                py_list.append('a')
                py_dic['a'] = len(py_list) - 1
                py_hz_dic['a'] = hzs[0]
            else:
                py_list.append(py)
                py_dic[py] = len(py_list) - 1
                py_hz_dic[py] = hzs[0]
            j += 1

            hz = hz[:-1]
            for hanzi in hz:
                if(hz_py_dic.__contains__(hanzi) == False):
                    i += 1
                    hz_py_dic[hanzi] = []
                    hz_py_dic[hanzi].append(i)
                    hz_list.append(hanzi)
                hz_py_dic[hanzi].append(py)
    
    hz_py_dic['阿'][1] =hz_py_dic['啊'][1] = hz_py_dic['呵'][1] = hz_py_dic['锕'][1] =  hz_py_dic['吖'][1]=  hz_py_dic['腌'][1]=  hz_py_dic['嗄'][1]='a'
    return hz_py_dic, hz_list, py_list, py_dic, py_hz_dic

In [2]:
def train_params():
    '''
    hmm参数训练
    :return:
    '''
    dic, hzl, pyl, pyd, pyhzdic = get_hz_py_dic()
    hz_num = len(hzl)
    pi = [0 for i in range(8000)]
    pi = count_pi(dic, pi, hzl, hz_num)

    b = [[0 for aa in range(8000)] for bb in range(8000)]
    b = count_b(hzl, dic, b, pyd, pyl)

    a = [[0 for aaa in range(8000)] for bbb in range(8000)]
    a = count_a(dic, a)
    return a, b, pi, dic, hzl, pyl, pyd, pyhzdic

In [3]:
def count_pi(dic, pi, hz_list, i):
    file = "lexicon.txt"
    with open(file, 'r', encoding='utf8') as fin:
        for line in fin.readlines():
            lineset = line.strip('\n').split(' ')
            hanzi = lineset[0][0]
            py = lineset[1][:-1]
            if(dic.__contains__(hanzi) == False):
                i += 1
                dic[hanzi] = []
                dic[hanzi].append(i)
                dic[hanzi].append(py)
                hz_list.append(hanzi)
            pi[dic[hanzi][0]] = 1
    return pi

In [4]:
def count_a(hz_dic, a):
    file = "lexicon.txt"
    count_a = [0 for i in range(8000)]
    with open(file, 'r', encoding='utf8') as fin:
        for line in fin.readlines():
            lineset = line.strip('\n').split(' ')
            Q = lineset[0]
            for q in range(len(Q) - 1):
                qi = hz_dic[Q[q]][0]
                qj = hz_dic[Q[q + 1]][0]
                a[qi][qj] += 1
                count_a[qi] += 1

    for i in range(len(a)):
        for j in range(len(a[0])):
            if(count_a[i] == 0):
                count_a[i] += 1
            a[i][j] = a[i][j] / count_a[i]
    return a


In [28]:
def count_b(hzlist, dic, b, pydic, pylist):
    file = "lexicon.txt"
    tmplist = [0 for i in range(8000)]
    with open(file, 'r', encoding='utf8') as fin:
        for line in fin.readlines():
            lineset = line.strip('\n').split(' ')
            hanzi = lineset[0]
            for hz in range(len(hanzi)):
                py = lineset[hz + 1][:-1]
                if(dic.__contains__(hanzi[hz]) == False):
                    hzlist.append(hanzi[hz])
                    dic[hanzi[hz]] = []
                    dic[hanzi[hz]].append(len(hzlist) - 1)
                    dic[hanzi[hz]].append(py)

                j = dic[hanzi[hz]][0] #汉字序列
                if(pydic.__contains__(py) == False):
                    pylist.append(py)
                    pydic[py] = len(pylist) - 1
                k = pydic[py]
                b[j][k] += 1
                tmplist[j] += 1

    for i in range(len(b)):
        for j in range(len(b[0])):
            b[i][j] = b[i][j] / (tmplist[i] + 1) ##smooth
    return b


In [29]:
a, b, pi, dic, hzl, pyl, pyd, pyhzdic = train_params()

In [45]:
def print_AB(a, hzl):
    for i in range(len(a)):
        for j in range(len(a[0])):
            if a[i][j] > 0:
                print(str(hzl[i]) + " " + str(hzl[j]) + " :" + str(a[i][j]))

In [47]:
print_AB(b, hzl)

阿 阿 :0.9636363636363636
阿 耙 :0.01818181818181818
啊 阿 :0.9090909090909091
呵 扳 :0.8888888888888888
爱 啊 :0.9857142857142858
暧 啊 :0.5
哀 啊 :0.9333333333333333
挨 啊 :0.9
埃 啊 :0.9411764705882353
诶 啊 :0.16666666666666666
诶 册 :0.6666666666666666
唉 啊 :0.75
隘 啊 :0.6666666666666666
艾 啊 :0.9
哎 啊 :0.8571428571428571
癌 啊 :0.9411764705882353
蔼 啊 :0.5
矮 啊 :0.8888888888888888
碍 啊 :0.875
皑 啊 :0.6666666666666666
安 呵 :0.9940476190476191
按 呵 :0.9565217391304348
桉 呵 :0.6666666666666666
氨 呵 :0.9166666666666666
铵 呵 :0.8333333333333334
庵 呵 :0.6666666666666666
俺 呵 :0.6666666666666666
鹌 呵 :0.5
胺 呵 :0.9
鞍 呵 :0.8571428571428571
案 呵 :0.9836065573770492
暗 呵 :0.96875
黯 呵 :0.6666666666666666
岸 呵 :0.9615384615384616
昂 锕 :0.9375
盎 锕 :0.75
肮 锕 :0.5
奥 吖 :0.9666666666666667
澳 吖 :0.8
懊 吖 :0.75
熬 吖 :0.8
傲 吖 :0.8571428571428571
嗷 吖 :0.75
敖 吖 :0.5
遨 吖 :0.5
鳌 吖 :0.5
袄 吖 :0.75
凹 吖 :0.8
坳 吖 :0.5
拗 吖 :0.25
拗 篦 :0.5
嚣 博 :0.8
巴 腌 :0.9873417721518988
爸 腌 :0.875
把 腌 :0.9655172413793104
吧 腌 :0.8888888888888888
靶 腌 :0.8
耙 腌 :0.33333333333

吃 岸 :0.9722222222222222
齿 岸 :0.9411764705882353
持 岸 :0.975
池 岸 :0.9666666666666667
弛 岸 :0.5
驰 岸 :0.9
迟 岸 :0.9285714285714286
赤 岸 :0.9655172413793104
哧 岸 :0.5
痴 岸 :0.8333333333333334
耻 岸 :0.875
翅 岸 :0.8333333333333334
斥 岸 :0.9375
叱 岸 :0.5
匙 岸 :0.3333333333333333
匙 滨 :0.3333333333333333
炽 岸 :0.8333333333333334
茌 岸 :0.5
侈 岸 :0.75
冲 昂 :0.9807692307692307
种 昂 :0.009345794392523364
种 蚕 :0.9813084112149533
忡 昂 :0.6666666666666666
崇 昂 :0.9166666666666666
虫 昂 :0.9807692307692307
充 昂 :0.96875
宠 昂 :0.8333333333333334
重 昂 :0.22297297297297297
重 蚕 :0.7702702702702703
憧 昂 :0.5
涌 补 :0.875
舂 昂 :0.5
抽 盎 :0.967741935483871
愁 盎 :0.9166666666666666
瞅 盎 :0.6666666666666666
丑 盎 :0.9090909090909091
筹 盎 :0.9523809523809523
畴 盎 :0.75
踌 盎 :0.6666666666666666
绸 盎 :0.9
稠 盎 :0.75
仇 盎 :0.8461538461538461
仇 遍 :0.07692307692307693
臭 盎 :0.8571428571428571
臭 帛 :0.07142857142857142
酬 盎 :0.8888888888888888
出 肮 :0.9961538461538462
础 肮 :0.9166666666666666
黜 肮 :0.5
刍 肮 :0.5
雏 肮 :0.8
处 肮 :0.9882352941176471
初 肮 :0.9761904761

风 叭 :0.9950248756218906
疯 叭 :0.8
讽 叭 :0.8
砜 叭 :0.5
枫 叭 :0.75
封 叭 :0.975609756097561
丰 叭 :0.9714285714285714
冯 叭 :0.75
峰 叭 :0.96
锋 叭 :0.9545454545454546
蜂 叭 :0.95
烽 叭 :0.6666666666666666
逢 叭 :0.9090909090909091
缝 叭 :0.95
奉 叭 :0.9583333333333334
俸 叭 :0.5
凤 叭 :0.9230769230769231
佛 拔 :0.9142857142857143
佛 魃 :0.05714285714285714
否 跋 :0.8947368421052632
否 裨 :0.05263157894736842
富 魃 :0.9807692307692307
副 魃 :0.9782608695652174
福 拔 :0.02127659574468085
福 魃 :0.9574468085106383
幅 魃 :0.9411764705882353
辐 魃 :0.8888888888888888
蝠 魃 :0.5
匐 魃 :0.5
抚 魃 :0.9411764705882353
赴 魃 :0.9090909090909091
讣 魃 :0.5
夫 魃 :0.9885057471264368
扶 魃 :0.9333333333333333
肤 魃 :0.8333333333333334
呋 魃 :0.5
芙 魃 :0.5
麸 魃 :0.6666666666666666
负 魃 :0.975609756097561
复 魃 :0.9900990099009901
腹 魃 :0.9411764705882353
馥 魃 :0.5
覆 魃 :0.9166666666666666
服 魃 :0.9861111111111112
付 魃 :0.9655172413793104
府 魃 :0.9761904761904762
附 魃 :0.9722222222222222
俯 魃 :0.875
驸 魃 :0.5
符 魃 :0.9411764705882353
浮 魃 :0.9655172413793104
俘 魃 :0.8333333333333334

魂 瓣 :0.8571428571428571
或 颁 :0.9285714285714286
惑 颁 :0.9
活 颁 :0.99
获 颁 :0.96
货 颁 :0.9879518072289156
火 颁 :0.99375
伙 颁 :0.9411764705882353
霍 颁 :0.9230769230769231
嚯 颁 :0.5
攉 颁 :0.5
祸 颁 :0.9285714285714286
几 扮 :0.9807692307692307
机 扮 :0.9971098265895953
饥 扮 :0.9090909090909091
矶 扮 :0.6666666666666666
叽 扮 :0.6666666666666666
肌 扮 :0.875
讥 扮 :0.75
及 扮 :0.967741935483871
圾 扮 :0.8333333333333334
极 扮 :0.9866666666666667
汲 扮 :0.5
级 扮 :0.9907407407407407
己 扮 :0.9545454545454546
记 扮 :0.9850746268656716
纪 扮 :0.984375
鸡 扮 :0.9583333333333334
基 扮 :0.9923076923076923
计 扮 :0.9913793103448276
集 扮 :0.9895833333333334
际 扮 :0.9838709677419355
齐 逼 :0.9642857142857143
济 扮 :0.9886363636363636
挤 扮 :0.9
剂 扮 :0.9807692307692307
跻 扮 :0.6666666666666666
荠 扮 :0.5
吉 扮 :0.9736842105263158
咭 扮 :0.5
继 扮 :0.9696969696969697
系 扮 :0.021739130434782608
系 铂 :0.967391304347826
急 扮 :0.9818181818181818
积 扮 :0.9868421052631579
季 扮 :0.96
悸 扮 :0.5
籍 扮 :0.9444444444444444
藉 扮 :0.8
技 扮 :0.98
伎 扮 :0.6666666666666666
妓 扮 :0.8
击 扮 :0

老 剥 :0.9940119760479041
姥 剥 :0.75
佬 剥 :0.5
劳 剥 :0.9901960784313726
捞 剥 :0.8333333333333334
涝 剥 :0.9
唠 剥 :0.6666666666666666
痨 剥 :0.5
牢 剥 :0.9375
烙 剥 :0.6
烙 辈 :0.2
络 辈 :0.9230769230769231
酪 剥 :0.8333333333333334
了 趵 :0.2727272727272727
了 碚 :0.7045454545454546
叻 趵 :0.5
肋 鸨 :0.75
勒 趵 :0.8780487804878049
勒 鸨 :0.07317073170731707
勒 婊 :0.024390243902439025
乐 趵 :0.42
乐 埔 :0.57
垒 鸨 :0.9090909090909091
类 鸨 :0.9827586206896551
雷 鸨 :0.9787234042553191
蕾 鸨 :0.875
擂 鸨 :0.6666666666666666
镭 鸨 :0.6666666666666666
累 鸨 :0.96
嘞 鸨 :0.5
泪 鸨 :0.9411764705882353
磊 鸨 :0.5
儡 鸨 :0.5
愣 被 :0.6666666666666666
楞 被 :0.5
棱 被 :0.9090909090909091
冷 被 :0.9814814814814815
丽 陂 :0.9565217391304348
鲡 陂 :0.5
厉 陂 :0.9
励 陂 :0.8888888888888888
雳 陂 :0.5
砺 陂 :0.5
力 陂 :0.9967105263157895
历 陂 :0.9818181818181818
沥 陂 :0.8333333333333334
砾 陂 :0.8
里 陂 :0.9923664122137404
理 陂 :0.9952153110047847
哩 陂 :0.75
喱 陂 :0.5
狸 陂 :0.5
鲤 陂 :0.5
厘 陂 :0.75
娌 陂 :0.5
李 陂 :0.9565217391304348
利 陂 :0.9947916666666666
俐 陂 :0.5
猁 陂 :0.5
梨 陂 :0.875
犁 陂 :0.8

农 必 :0.9911504424778761
浓 必 :0.9444444444444444
侬 必 :0.5
脓 必 :0.8
奴 秘 :0.9565217391304348
怒 秘 :0.9473684210526315
努 秘 :0.8333333333333334
暖 铋 :0.9444444444444444
挪 笔 :0.8
诺 笔 :0.9583333333333334
喏 笔 :0.5
糯 笔 :0.6666666666666666
懦 笔 :0.75
女 曹 :0.9888888888888889
虐 漕 :0.75
疟 漕 :0.6666666666666666
噢 濞 :0.5
区 敝 :0.01834862385321101
区 鳊 :0.9724770642201835
欧 敝 :0.9629629629629629
瓯 敝 :0.6666666666666666
鸥 敝 :0.5
殴 敝 :0.75
沤 敝 :0.5
呕 敝 :0.75
讴 敝 :0.6666666666666666
偶 敝 :0.9285714285714286
耦 敝 :0.5
藕 敝 :0.5
怕 弊 :0.9090909090909091
帕 弊 :0.8571428571428571
啪 弊 :0.6666666666666666
爬 弊 :0.8888888888888888
筢 弊 :0.5
琶 弊 :0.5
趴 弊 :0.5
派 辟 :0.98
牌 辟 :0.9705882352941176
排 辟 :0.9807692307692307
徘 辟 :0.5
湃 辟 :0.5
拍 辟 :0.9615384615384616
迫 辟 :0.10714285714285714
迫 弼 :0.8571428571428571
跑 碧 :0.967741935483871
泡 碧 :0.96
袍 碧 :0.75
咆 碧 :0.5
狍 碧 :0.5
疱 碧 :0.5
抛 碧 :0.8888888888888888
盘 璧 :0.9807692307692307
潘 璧 :0.75
判 璧 :0.975
畔 璧 :0.75
胖 壁 :0.8333333333333334
攀 璧 :0.875
拚 庳 :0.6666666666666666
盼 璧 :0.8
叛 璧 :

髓 杓 :0.8571428571428571
穗 杓 :0.9411764705882353
遂 杓 :0.8
隧 杓 :0.6666666666666666
燧 杓 :0.5
碎 杓 :0.9473684210526315
绥 杓 :0.75
祟 杓 :0.75
孙 彪 :0.9615384615384616
荪 彪 :0.5
损 彪 :0.9411764705882353
笋 彪 :0.8888888888888888
所 髟 :0.9886363636363636
索 髟 :0.972972972972973
嗦 髟 :0.6666666666666666
唢 髟 :0.5
锁 髟 :0.9230769230769231
琐 髟 :0.8
梭 髟 :0.8571428571428571
羧 髟 :0.6666666666666666
唆 髟 :0.75
莎 飙 :0.8
娑 髟 :0.5
沙 飙 :0.9848484848484849
纱 飙 :0.96
砂 飙 :0.9545454545454546
痧 飙 :0.5
鲨 飙 :0.5
裟 飙 :0.5
杀 飙 :0.9772727272727273
啥 飙 :0.875
厦 飙 :0.5555555555555556
厦 泊 :0.3333333333333333
傻 飙 :0.75
煞 飙 :0.8333333333333334
霎 飙 :0.5
杉 飙 :0.125
杉 飑 :0.75
晒 镳 :0.9
筛 镳 :0.9
山 飑 :0.9960159362549801
汕 飑 :0.5
讪 飑 :0.5
舢 飑 :0.5
善 飑 :0.9666666666666667
膳 飑 :0.6666666666666666
缮 飑 :0.5
陕 飑 :0.8
闪 飑 :0.9583333333333334
扇 飑 :0.9
煽 飑 :0.75
骟 飑 :0.5
衫 飑 :0.9285714285714286
珊 飑 :0.8333333333333334
删 飑 :0.8
姗 飑 :0.6666666666666666
跚 飑 :0.5
栅 飑 :0.5
栅 睬 :0.25
擅 飑 :0.75
赡 飑 :0.6666666666666666
上 别 :0.9969788519637462
商 别 :0.992

侮 簸 :0.75
巫 簸 :0.8
诬 簸 :0.8
戊 簸 :0.75
兀 簸 :0.6666666666666666
西 铂 :0.9939393939393939
牺 铂 :0.8333333333333334
奚 铂 :0.5
溪 铂 :0.9230769230769231
喜 铂 :0.9787234042553191
禧 铂 :0.5
嬉 铂 :0.75
嘻 铂 :0.8571428571428571
熹 铂 :0.5
息 铂 :0.9807692307692307
熄 铂 :0.8
媳 铂 :0.8571428571428571
螅 铂 :0.6666666666666666
洗 铂 :0.95
洗 舶 :0.025
细 铂 :0.9859154929577465
锡 铂 :0.9375
希 铂 :0.9523809523809523
稀 铂 :0.9523809523809523
烯 铂 :0.9375
浠 铂 :0.5
席 铂 :0.9714285714285714
习 铂 :0.972972972972973
吸 铂 :0.972972972972973
袭 铂 :0.9230769230769231
析 铂 :0.96
晰 铂 :0.75
淅 铂 :0.6666666666666666
熙 铂 :0.8
昔 铂 :0.8
惜 铂 :0.875
悉 铂 :0.8888888888888888
夕 铂 :0.9
矽 铂 :0.8333333333333334
汐 铂 :0.5
膝 铂 :0.8571428571428571
玺 铂 :0.5
曦 铂 :0.5
羲 铂 :0.5
隙 铂 :0.875
犀 铂 :0.75
铣 铂 :0.5
铣 舶 :0.25
徙 铂 :0.5
蟋 铂 :0.5
檄 铂 :0.5
樨 铂 :0.5
下 泊 :0.996309963099631
夏 泊 :0.9655172413793104
峡 泊 :0.9166666666666666
侠 泊 :0.8888888888888888
狭 泊 :0.8571428571428571
辖 泊 :0.875
瞎 泊 :0.8571428571428571
瑕 泊 :0.5
暇 泊 :0.8
遐 泊 :0.6666666666666666
霞 泊 :0.92857142857

玉 哺 :0.9767441860465116
鱼 哺 :0.9864864864864865
渔 哺 :0.9583333333333334
域 哺 :0.9375
雨 哺 :0.9838709677419355
誉 哺 :0.9333333333333333
予 哺 :0.9090909090909091
豫 哺 :0.8571428571428571
预 哺 :0.9824561403508771
语 哺 :0.9873417721518988
龉 哺 :0.5
裕 哺 :0.8571428571428571
浴 哺 :0.9333333333333333
峪 哺 :0.5
欲 哺 :0.95
俞 哺 :0.5
喻 哺 :0.8333333333333334
愉 哺 :0.6666666666666666
渝 哺 :0.6666666666666666
榆 哺 :0.6666666666666666
瑜 哺 :0.6666666666666666
谕 哺 :0.6666666666666666
觎 哺 :0.5
愈 哺 :0.9090909090909091
逾 哺 :0.75
郁 哺 :0.9230769230769231
禹 哺 :0.75
娱 哺 :0.8
羽 哺 :0.9375
禺 哺 :0.5
隅 哺 :0.6666666666666666
愚 哺 :0.9
遇 哺 :0.9565217391304348
寓 哺 :0.8888888888888888
御 哺 :0.875
狱 哺 :0.9090909090909091
淤 哺 :0.75
瘀 哺 :0.75
煜 哺 :0.5
舆 哺 :0.6666666666666666
熨 逋 :0.6666666666666666
驭 哺 :0.5
粥 灿 :0.6666666666666666
元 晡 :0.9866666666666667
沅 晡 :0.5
远 晡 :0.9855072463768116
园 晡 :0.9803921568627451
原 晡 :0.9924242424242424
源 晡 :0.9811320754716981
愿 晡 :0.967741935483871
员 晡 :0.9884393063583815
员 逋 :0.005780346820809248
圆 晡 :0.9

In [56]:
def print_pi(pi, hzl):
    for i in range(len(pi)):
            print(str(hzl[i]) + " " + str(pi[i]))

In [57]:
print_pi(pi,hzl)

阿 1
啊 1
呵 1
锕 0
吖 0
腌 0
嗄 0
爱 1
嫒 0
暧 1
瑷 0
哀 1
锿 0
挨 1
埃 1
诶 1
唉 1
隘 1
嗌 0
艾 1
哎 1
砹 0
癌 1
蔼 0
霭 0
矮 1
碍 1
皑 1
安 1
按 1
桉 1
氨 1
铵 1
庵 1
俺 1
鹌 1
埯 0
胺 0
鞍 1
案 1
暗 1
揞 0
黯 1
谙 0
岸 1
昂 1
盎 1
肮 1
奥 1
澳 1
懊 1
熬 1
傲 1
嗷 1
敖 1
廒 0
遨 1
鳌 0
骜 0
螯 0
獒 0
岙 0
袄 1
凹 1
坳 0
拗 1
嚣 1
媪 0
巴 1
岜 0
爸 1
把 1
吧 1
靶 1
耙 1
芭 1
笆 1
疤 1
粑 1
钯 0
八 1
扒 1
叭 1
拔 1
跋 1
魃 0
茇 0
霸 1
灞 0
罢 1
捌 1
坝 1
白 1
伯 1
柏 1
百 1
佰 1
捭 0
稗 1
败 1
呗 1
拜 1
摆 1
掰 1
半 1
伴 1
绊 1
拌 1
版 1
板 1
扳 1
坂 0
阪 0
钣 0
舨 0
般 1
搬 1
瘢 1
班 1
斑 1
癍 0
办 1
瓣 1
颁 1
扮 1
邦 1
绑 1
梆 1
帮 1
榜 1
傍 1
磅 1
膀 1
镑 1
谤 0
蒡 0
棒 1
浜 0
蚌 1
包 1
饱 1
刨 1
胞 1
炮 1
苞 1
鲍 1
龅 0
雹 1
报 1
保 1
褓 0
葆 1
堡 1
煲 1
褒 1
宝 1
薄 1
豹 1
暴 1
爆 1
剥 1
趵 1
鸨 1
被 1
陂 0
倍 1
焙 1
碚 0
蓓 1
北 1
邶 0
背 1
褙 0
臂 1
杯 1
贝 1
狈 0
钡 0
辈 1
悲 1
卑 1
碑 1
萆 0
悖 1
鹎 0
备 1
惫 1
鞴 1
鐾 0
本 1
苯 1
笨 1
奔 1
贲 1
锛 0
坌 0
夯 1
畚 0
崩 1
绷 1
蹦 1
嘣 1
泵 1
甭 1
迸 1
甏 0
匕 1
比 1
毙 1
秕 0
妣 0
吡 0
庇 1
毖 0
毕 1
陛 1
篦 0
蓖 0
必 1
秘 1
铋 0
笔 1
滗 0
鼻 1
濞 0
敝 1
弊 1
辟 1
碧 1
璧 1
壁 1
避 1
襞 0
畀 0
裨 1
婢 0
俾 1
髀 0
庳 0
痹 0
弼 0
蔽 1
币 1
逼 1
彼 1
闭 1
拂 1
愎 0
鄙 1
边 1


莽 1
蟒 0
漭 0
盲 1
麽 0
每 1
梅 1
酶 1
莓 0
霉 1
没 1
美 1
镁 1
煤 1
媒 1
枚 1
妹 1
昧 1
魅 1
眉 1
湄 1
媚 1
楣 1
猸 0
鹛 0
嵋 0
镅 0
玫 1
寐 1
袂 0
糜 1
谜 1
浼 0
门 1
们 1
闷 1
扪 0
钔 0
焖 1
懑 0
孟 1
猛 1
锰 1
蜢 0
勐 0
艋 0
梦 1
蒙 1
檬 0
朦 1
盟 1
萌 1
懵 0
米 1
迷 1
咪 1
眯 1
脒 0
醚 0
敉 0
麋 1
泌 1
谧 0
宓 0
密 1
蜜 1
嘧 0
弥 1
祢 0
猕 0
靡 0
縻 0
蘼 0
芈 0
觅 1
弭 0
幂 0
苗 1
瞄 1
描 1
喵 1
鹋 0
庙 1
秒 1
妙 1
杪 0
眇 1
渺 1
缈 0
缪 0
淼 0
邈 0
藐 1
面 1
缅 1
腼 1
湎 0
棉 1
绵 1
免 1
勉 1
娩 1
冕 0
沔 0
眄 0
眠 1
黾 0
渑 0
灭 1
蔑 1
篾 1
蠛 0
咩 1
乜 0
民 1
泯 1
珉 0
岷 1
抿 1
苠 0
缗 0
敏 1
玟 0
闽 1
闵 1
悯 0
愍 0
鳘 0
皿 0
名 1
铭 1
酩 0
茗 1
明 1
鸣 1
命 1
冥 1
溟 0
瞑 0
暝 0
螟 0
谬 1
末 1
沫 1
秣 0
摸 1
莫 1
模 1
膜 1
馍 1
谟 0
漠 1
镆 0
嫫 0
貘 0
瘼 0
寞 0
蓦 1
摹 1
殁 0
墨 1
耱 0
磨 1
蘑 1
魔 1
默 1
万 1
茉 1
陌 1
貊 0
无 1
某 1
谋 1
牟 1
眸 1
哞 1
侔 0
蛑 0
鍪 0
木 1
亩 1
牧 1
母 1
姆 0
拇 1
坶 0
目 1
钼 0
苜 1
幕 1
墓 1
慕 1
募 1
暮 1
沐 1
穆 1
睦 1
牡 1
仫 0
毪 0
那 1
哪 1
娜 1
拿 1
镎 0
纳 1
钠 1
呐 1
衲 0
肭 0
捺 1
呢 1
南 1
耐 1
乃 1
奶 1
艿 0
氖 0
奈 1
柰 0
萘 1
鼐 0
脑 1
恼 1
垴 0
瑙 1
硇 0
闹 1
铙 0
挠 1
蛲 0
淖 0
孬 1
猱 0
呶 1
楠 1
喃 1
蝻 0
腩 0
难 1
男 1
囡 1
赧 0
囊 1
曩 0
馕 1
囔 0
攮 0
讷 0
内 1


作 1
昨 1
柞 1
祚 0
怍 0
胙 0
阼 0
做 1
坐 1
唑 0
座 1
左 1
佐 1
琢 1
札 1
乍 1
炸 1
诈 1
蚱 0
砟 0
痄 0
榨 1
渣 1
揸 0
齄 0
闸 1
咤 0
眨 1
铡 1
吒 0
哳 0
寨 1
债 1
摘 1
翟 0
瘵 0
砦 0
窄 1
宅 1
斋 1
占 1
站 1
沾 1
战 1
毡 1
展 1
搌 0
辗 1
湛 1
詹 1
瞻 1
谵 0
斩 1
崭 1
栈 1
盏 1
蘸 0
绽 1
旃 0
张 1
账 1
胀 0
帐 1
涨 1
章 1
蟑 0
障 1
漳 1
樟 1
璋 0
幛 0
獐 0
嫜 0
彰 0
鄣 0
瘴 0
仉 0
掌 1
丈 1
仗 1
杖 1
找 1
赵 1
着 1
照 1
兆 1
昭 1
招 1
沼 1
诏 1
罩 1
棹 0
肇 1
钊 1
爪 1
笊 0
啁 0
这 1
者 1
锗 0
赭 0
浙 1
哲 1
蜇 0
蛰 0
蔗 1
鹧 0
遮 1
辙 0
柘 0
辄 0
谪 0
磔 0
褶 1
真 1
镇 1
缜 0
稹 0
针 1
甄 0
振 1
赈 1
震 1
珍 1
诊 1
疹 1
轸 0
胗 0
畛 0
阵 1
贞 1
桢 0
侦 1
祯 0
浈 0
帧 1
圳 0
枕 1
臻 0
榛 0
蓁 0
斟 1
箴 1
鸩 0
朕 1
砧 1
正 1
证 1
政 1
征 1
怔 0
钲 0
整 1
症 1
郑 1
争 1
挣 1
睁 1
铮 1
峥 1
诤 0
狰 1
筝 0
蒸 1
拯 1
至 1
侄 1
桎 0
轾 0
蛭 0
致 1
窒 1
膣 0
之 1
芝 1
只 1
职 1
织 1
帜 0
轵 0
枳 0
制 1
志 1
支 1
枝 1
忮 0
肢 1
治 1
直 1
值 1
植 1
埴 1
置 1
质 1
踬 0
纸 1
旨 1
酯 1
指 1
脂 1
知 1
蜘 1
智 1
执 1
止 1
址 0
趾 0
祉 0
芷 0
滞 1
郅 0
汁 1
掷 1
踯 1
炙 0
稚 1
痔 1
雉 0
挚 1
秩 1
栉 0
痣 1
彘 0
骘 0
贽 0
鸷 0
帙 0
黹 0
豸 0
跖 0
絷 0
徵 0
摭 0
咫 0
祗 0
胝 0
觯 0
卮 0
栀 0
瘛 0
中 1
忠 1
钟 1
肿 1
仲 1
舯 0
盅 0
众 1
锺 0
踵 0


IndexError: list index out of range

In [30]:
def viterbi(obs, pyd, length, pydic, pi, a, b, hzdic, hzl):
    obs += ' a'
    OBS = obs.split(" ")
    T = len(OBS)
    N = length
    idx = pyd[OBS[0]]
    eta = [[0 for js in range(N)] for it in range(T)]
    phi = [[0 for js in range(N)] for it in range(T)]
    ### initialize
    pre_py = pydic[OBS[0]]
    for item in range(len(pre_py)):
        hzidx = hzdic[pre_py[item]][0]
        eta[0][item] = pi[hzidx]*b[hzidx][idx]

    for t in range(1, T):
        currO = OBS[t] # 这是一个拼音
        idxo = pyd[currO]
        ot_py = pydic[currO] #当前状态的拼音对应的汉字集
        for jj in range(len(ot_py)):
            j = ot_py[jj] #当前汉字
            ot_pre_py = pydic[OBS[t - 1]] #前一个状态的所有汉字对应
            idx_j = hzdic[j][0]
            maxn = -1 #
            argmaxn = 0
            for ii in range(len(ot_pre_py)):
                i = ot_pre_py[ii]
                idx_i = hzdic[i][0]
                if maxn < eta[t-1][ii] * a[idx_i][idx_j]:
                    maxn = eta[t-1][ii] * a[idx_i][idx_j]
                    argmaxn = idx_i
            eta[t][jj] = maxn * b[idx_j][idxo]
            phi[t][jj] = argmaxn
    
    ##接下来是做回溯的部分
    word_result = ""
    for i in range(1, len(phi)):
        idxt = max(phi[i])
        word_result +=hzl[idxt]
    return word_result

In [31]:
def transfer(a, b, pi, dic, hzl, pyl, pyd, pyhzdic):
    a_new = dict()
    b_new = dict()
    pi_new = dict()
    for i in range(len(a)):
        a_new[hzl[i]] = dict()
        for j in range(len(a[0])):
            a_new[hzl[i]][hzl[j]] = a[i][j]
            
    for j in range(len(b)):
        b_new[hzl[i]] = dict()
        for j in range(len(b[0])):
            b_new[hzl[i]][pyl[j]] = b[i][j]
            
    for i in range(len(pi)):
        pi_new[hzl[i]] = pi[i]
    return a_new, b_new, pi_new 

In [32]:
length = 0
for i in pyhzdic.values():
    length = max(length, len(i))
print(length)

111


In [38]:
def display():
    count_all = 0
    count_true = 0
    file = "test.txt"
    now_chinese = []
    test_result = []
    with open(file, 'r', encoding='utf8') as fin:
        i = 0
        for line in fin.readlines():
            line = line.strip('\n')
            if(i % 2 != 0):
                now_chinese.append(line)
            else:
                phi = viterbi(line, pyd, length,pyhzdic, pi, a, b, dic, hzl)
                test_result.append(phi)
            i+=1

    for j in range(len(now_chinese)):
        count_all += len(now_chinese[j])
        print("原文：" + now_chinese[j])
        print("训练结果：" + test_result[j])
        for x in range(len(now_chinese[j])):
            if now_chinese[j][x] == test_result[j][x]:
                count_true +=1
    
    print("正确率：" + str(count_true/count_all))

In [39]:
display()

原文：今天晚上有好看的电影
训练结果：今天晚上有好看的电映
原文：北京奥运会开幕式非常精彩
训练结果：备京奥运会开幕式非常精才
原文：全国人民代表大会在北京人民大会堂隆重召开
训练结果：全国人民代表大会在北京人民大会堂隆中找开
原文：金庸的武侠小说非常精彩
训练结果：今庸得武侠小说非常精才
原文：你的世界会变得更精彩
训练结果：尼的世界会变得更精才
原文：在天安门广场举行了隆重的阅兵式
训练结果：在天安门广场举省了隆中的阅兵是
原文：逻辑代数有些常用的基本公式
训练结果：逻辑代数有些常用得基本公是
原文：请大家选择你觉得可以的时间
训练结果：卿达家选择你决得可以的时件
原文：具有良好的沟通能力和交流能力
训练结果：具有良好的沟通能力和交流能丽
原文：人在阅读时是从左到右逐字读入的
训练结果：人在阅渎时是从左到右逐字渎入的
原文：减少课程的课时是十分科学的
训练结果：减少课程的课时事事分科学的
原文：我戴着一副黑框眼镜
训练结果：我戴这一副黑框眼竞
原文：他钢琴弹得不错
训练结果：他钢琴摊得不错
原文：小朋友们都喜欢去郊游
训练结果：小朋友们都喜欢去郊有
原文：这个玩具很有趣
训练结果：遮个玩具很有区
原文：他很生气
训练结果：他很生气
原文：今天天气很好
训练结果：今天天气很好
原文：我们去散步吧
训练结果：我们去散步巴
原文：大家可能不知道
训练结果：打家可能不知岛
原文：我们宿舍的灯坏了
训练结果：我们宿舍的灯坏了
正确率：0.8428571428571429
